In [ ]:
import json
import requests
import time
import os
import logging
from datetime import datetime

In [ ]:
OPENSEA_API_BASE = "https://api.opensea.io/api/v2"
OPENSEA_API_KEY = "" 
OPENSEA_API_BASE_V1 = "https://api.opensea.io/api/v1" 
collection_slug="doodles-official" # [pudgypenguins, chimpersnft, doodles-official]
chain="ethereum"

In [ ]:
get_nft_event_url = "https://api.opensea.io/api/v2/events/chain/{chain}/contract/{contract_address}/nfts/{identifier}/?event_type=sale&next={next_string}"


In [ ]:
def make_api_request(url, headers=None, params=None):
    """Make API request with basic rate limiting"""
    max_retries = 3
    retry_count = 0
    
    if headers is None:
        headers = {
            "Accept": "application/json",
            "X-API-KEY": OPENSEA_API_KEY
        }
    while True:
        try:
            response = requests.get(url, headers=headers, params=params)
            
            # Handle rate limiting
            if response.status_code == 429:
                print("Rate limited, waiting 1 second...")
                time.sleep(1)
                return make_api_request(url, headers, params)
            
            # Raise exception for other errors
            response.raise_for_status()
            retry_count = 0
            return response.json()
        except Exception as e:
            print(f"Error making request to {url}: {e}")
            retry_count += 1
            if retry_count >= max_retries:
                print(f"Max retries reached")
                return None
            time.sleep(2 ** retry_count)  # Exponential backoff


In [ ]:
def get_nfts_event(chain, contract_address, identifier,next_string=""):
    url = get_nft_event_url.format(chain=chain,contract_address=contract_address, identifier=identifier,next_string=next_string)
    return make_api_request(url)

In [ ]:
def fetch_all_nft_events(chain, contract_address, identifier):
    """Fetch all events for an NFT, handling pagination"""
    all_events = []
    next_string = ""
    retry_count = 0
    max_retries = 3
    
    while True:
        response = get_nfts_event(chain, contract_address, identifier, next_string=next_string)
            
        # Add new events to our collection
        if 'asset_events' in response:
            all_events.extend(response['asset_events'])
        
        # Check for more pages
        if 'next' in response and response['next']:
            next_string = response['next']
            time.sleep(1)  # Rate limiting
            retry_count = 0  # Reset retry counter on success
        else:
            break
    
    return all_events

In [ ]:

def requset_log(log_dir):
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(f"{log_dir}nft_events_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"),
            logging.StreamHandler()  # Also prints to console
        ]
    )

In [ ]:
pass_identifier = -1

In [ ]:
# Get all NFTs in the collection
nfts_by_collections_file = f"./data/{collection_slug}/nfts_by_collection_data_sorted.json"
nfts_event_output_dir= f"./data/{collection_slug}/sale_events/"
os.makedirs(nfts_event_output_dir, exist_ok=True)
requset_log(nfts_event_output_dir)

nfts_event_json=[]

with open(nfts_by_collections_file, 'r') as f:
    nfts_by_collections_json = json.load(f)
    for nft in nfts_by_collections_json:
        
        identifier= nft['identifier']
        if int(identifier) <= pass_identifier:
            continue
        contract_address= nft['contract']
        nft_event_json_name= f"{collection_slug}_{identifier}.json"
        
        all_nft_events = fetch_all_nft_events(chain, contract_address, identifier)
        # get_nfts_event_response = get_nfts_event(chain, contaract_address, identifier)
        # nft_events= get_nfts_event_response['asset_events']
        # nft_next= get_nfts_event_response['next']
        event_data = {
                "metadata": {
                    "collection_slug": collection_slug,
                    "contract": contract_address,
                    "identifier": identifier,
                    "total_events": len(all_nft_events)
                },
                "events": all_nft_events
            }
            
            # Save to file
        with open(nfts_event_output_dir+nft_event_json_name, 'w') as outfile:
            json.dump(event_data, outfile, indent=2)
        
        logging.info(f"Saved {len(all_nft_events)} events for NFT {identifier}")
        